In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy as d_copy
import random
from condconv import CondConv2D
from torch.utils.tensorboard import SummaryWriter

print("===INFO===")
print("torch ver : %s\ntorchvision ver : %s " %(torch.__version__, torchvision.__version__))
print("GPU model :",torch.cuda.get_device_name(0))


===INFO===
torch ver : 1.7.1
torchvision ver : 0.8.2 
GPU model : TITAN RTX


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter("./runs/3fc_model_5e-1_CondConv")
log_file = "./3fc_64_acc_log_0915.txt"
#error_index = 0
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)#.to(device)
vgg16_bn.eval()
print(device)

cuda


In [3]:
# randomness 제어 
# https://hoya012.github.io/blog/reproducible_pytorch/
def set_randomness(seed=0):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
# func

# only apply for feature part (not pooling, classfier)
# because of layers.feature 
def split_layer(model,start,end):
    ct = 0
    split_model=[] # from start to Conv5_1(include ReLU)
    for name,layers in model.named_modules():
        #print(name,layer)
        #print(layers.features)
        for idx,layer in enumerate(layers.features):
            #print(idx,layer)
            if start <=idx and idx <=end :
                split_model.append(layer)
        break
    return nn.Sequential(*split_model)

seed = 0
set_randomness(seed)

In [4]:

def get_dataset(num_train,batch_size,
                dataset_path,retrain_model_path):
    if os.path.isdir(retrain_model_path) is False:
        # make folder
        os.mkdir(retrain_model_path)
        print("retrain model path created :",os.listdir(retrain_model_path+"../"))
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    transforms_train = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])

    transforms_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])
    train_dataset = dataset.ImageFolder(root=dataset_path+"train",
                                       transform=transforms_train)
    subset_train_dataset,_ = torch.utils.data.random_split(train_dataset, 
                                        [num_train,len(train_dataset)-num_train])
    test_dataset = dataset.ImageFolder(root=dataset_path+"val",
                                       transform=transforms_test)
    
    train_dataloader = torch.utils.data.DataLoader(subset_train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=4) # for using subset
    test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=4)
    print("train dataset[%d], test dataset[%d] are loaded"%(len(train_dataloader),len(test_dataloader)))
    return train_dataloader,test_dataloader


dataset_path = "/media/2/Network/Imagenet_dup/"
retrain_model_path = "/media/0/Network/0821_to_fullmodels/"
batch_size = 128 # 32~ out of memory in 3080
num_train = 128000 #640000

train_dataloader,test_dataloader = get_dataset(num_train,batch_size,
                  dataset_path,retrain_model_path)

train dataset[1000], test dataset[391] are loaded


In [5]:
# external variable in error_index, num_error

def make_error_info(error_index, num_error):
    data = []
    #for i in range(511,-1,-1):
    for i in range(512):
        if error_index <= i and i < error_index+num_error:
            data.append(1)
        else :
            data.append(0)
        #print(data)
    error_info = torch.Tensor(data)
    error_info  = error_info.unsqueeze(0).repeat(512,1)
    #print("error_info :",error_info)
    return error_info # 512,521
class F4F(nn.Module):
    def __init__(self):
        super().__init__()
        #self.f4f = nn.Linear(3*3*512+512,3*3*512) # 5120,4608 filter which change feature.34 (Conv5_1)
        self.layer1 = CondConv2D(512,512,kernel_size=3,
                                 padding=(1,1),num_experts=10)
        self.layer2 = CondConv2D(512,512,kernel_size=3,
                                 padding=(1,1),num_experts=10)
        self.layer3 = CondConv2D(512,512,kernel_size=3,
                                 padding=(1,1),num_experts=10)
  
        # 512 x5120 사이즈로 batch 저장
    def get_f4f_weight(self):
        # fc.weight.size(),fc.bias.size()
        return self.parameters # torch.Size([4608, 5120])
    def forward(self,x):
        x1_ = self.layer1(x)
        x1  = torch.relu(x1_) # "nn.functional.tanh is deprecated. Use torch.tanh instead.")
        #print("x1",x1.size())
        x2_ = self.layer2(x1)
        x2  = torch.relu(x2_)
        #print("x2",x2.size())
        x3_ = self.layer3(x2)
        y   = torch.tanh(x3_)
        return y
        

In [6]:
error_index=0
num_error = 64
f4f = F4F().to(device)
optimizer = torch.optim.SGD(f4f.parameters(),lr=5e-1,weight_decay=1e-4)
if torch.cuda.device_count() >1 :
    print("data parallel start")
    f4f = nn.DataParallel(f4f).to(device)
def loss_fn(loss1_ratio,pred,label, filter_orig,filter_f4f):
    if(loss1_ratio <0 or loss1_ratio >1 ):
        print("wrong parameter ratio ",loss1_ratio)
        return nan
    loss1 = nn.CrossEntropyLoss()
    loss2 = nn.MSELoss()
    a = loss1(pred,label)
    b = loss2(filter_orig,filter_f4f)
    return loss1_ratio * a + (1-loss1_ratio) * b

In [7]:

def error_injection(name,num_error,error_index):
    def hook(model,input):
        start = error_index
        end = error_index + num_error
        #print(input.shape) #not working
        #normalize = nn.BatchNorm2d(512)
        input[0][:, start:end]=0
        #input = normalize(input)
        #print("error injection shape :",input[0][:, start:end],"original : ",input) # dbug
    return hook

def hook_register(model,num_error,error_index):
    for name,layers in model.named_modules():
        #print(name,layer)
        for idx,layer in enumerate(layers.features):
            #print(idx,layer)
            if idx is 34 and isinstance(layer, torch.nn.modules.conv.Conv2d) :
                print("input",name,layer) # target layer Conv5_1
                layer.register_forward_pre_hook(error_injection(name,num_error,error_index))
        break

In [8]:
class Target_model(nn.Module):
    def __init__(self,model):
        super().__init__()
        self.model = model
    def get_layer(self,idx):
        #print(self.model._modules['34'])
        layer =None
        try : # target model
            layer = self.model._modules[str(idx)]
        except KeyError: # test_model
            layer = self.model.features._modules[str(idx)]
        return layer
    def apply_f4f(self,f4f,error_info):
        weight = self.get_layer(34).weight.data
        #weight = torch.reshape(self.get_layer(34).weight.data,(512,512*3*3)).to(device) # flatten [512,4608] (batch 512)
        #data = torch.cat( (weight,error_info), 1 ) # 기존
        #data = torch.cat( (error_info,weight), 1 ) #210801 error_info를 앞에 붙이는 방법
        #offset = torch.reshape(f4f(data),(512,512,3,3))
        offset = f4f(weight)
        #self.get_layer(34).weight.data = self.get_layer(34).weight.data + offset
        self.get_layer(34).weight.data = offset
        #print(self.get_layer(34).weight)
    def forward(self,x,f4f,error_info):
        origin_weight = self.get_layer(34).weight.data
        # apply_f4f는 매 epoch마다 동일하므로 
        self.apply_f4f(f4f,error_info)
        replace_weight = self.get_layer(34).weight.data
        y = self.model(x)
        return y, origin_weight, replace_weight


In [9]:

original_model = d_copy(vgg16_bn).to(device)
hook_register(vgg16_bn,num_error,error_index)
target_model = Target_model(vgg16_bn).to(device)

input  Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [10]:
# evaluation phasetraining
def eval(model,dataloader,epoch,loss_fn,batch_size,
         f4f,error_info,log_file,TensorBoardWriter):
    
    
    model.cuda()
    model.eval()
    f4f.eval()
    total = 0
    correct =0
    total_loss =0.0
    with torch.no_grad():
        print("======eval start=======")
        for i, data in enumerate(dataloader):
            inputs,labels = data
            inputs,labels = inputs.cuda(), labels.cuda()
        
            #y_hat = model(inputs,f4f,error_info)
            result = model(inputs,f4f,error_info)
            y_hat,origin_weight, replace_weight = result
            
            _, predicted = torch.max(y_hat, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss = loss_fn(0.5,y_hat,labels,
                            origin_weight, replace_weight)
            total_loss +=loss.item()
            
            if(i%200 == 199):
                print("step : %d / %d acc : %.3f"
                      %(i + 1,int(len(dataloader)), correct*100/total))
                #print(".",end="")
        print("")
    acc = 100*correct/total
    #print(total_loss, len(dataloader))
    avg_loss = total_loss / (len(dataloader)*batch_size)
    print("Eval acc of model on imagenet : %.4f %%, Loss : %.4f" %(acc,avg_loss)) # model.__class__.__name__
    f = open(log_file,"a")
    print("Eval acc of model on imagenet : %.4f %%, Loss : %.4f" %(acc,avg_loss),file=f) # model.__class__.__name__
    f.close()
    TensorBoardWriter.add_scalar("Model1/ACC_EVAL",acc,epoch)
    TensorBoardWriter.add_scalar("Model1/LOSS_EVAL",avg_loss,epoch)
    print("======eval  end ======")  
    return acc


In [11]:
# training
def training(f4f,target_model,original_model,
             train_dataloader,test_dataloader,batch_size,
             log_file,TensorBoardWriter,
             retrain_model_path,
             loss_fn,optimizer,
             num_error,
             max_epochs=30,subset=False):
    first_feature = []
    first_label = []
    original_out = []
    offset_info = []
    target_model.to(device)
    original_model.to(device)
    target_model.eval()
    original_model.eval()
    
    feature_num = 100
    for epoch in range(1,max_epochs+1):
        running_loss = 0.0
        total_loss = []
        total_avg_loss = 0.0
        total = 0
        correct = 0
        f4f.train()
        # update f4f filter
        #target_model.apply_f4f(f4f,error_info)
    
        # compare
        for i, data in enumerate(train_dataloader):
            error_index = i % (512-num_error)
            error_info = make_error_info(error_index,num_error).to(device)
            
            
            if i % 10 == 0:
                print(".",end="")
            inputs,labels = data
            inputs,labels = inputs.to(device), labels.to(device)
            
            #target_out = target_model(inputs,f4f,error_info)
            result = target_model(inputs,f4f,error_info)
            target_out,origin_weight, replace_weight = result
            
            if len(first_feature) < feature_num:
                first_feature.append(target_out)
                first_label.append(labels)
                #first_feature.pop(0)
                #first_label.pop(0)
            _,predicted = torch.max(target_out,1) # target_out.data : no grad, target_out : with grad
            
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
            #loss = loss_fn(target_out,labels)
            
            loss = loss_fn(0.5,target_out,labels,
                            origin_weight, replace_weight) # check ratio is same of eval func
            
            running_loss += loss.item()
            #target_model.model.zero_grad() # might be useless
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 100 == 99: 
                total_loss.append(running_loss/100)
                print("")
                print('[%d, %5d] loss: %.6f' % (epoch+1, i, running_loss/(100*batch_size) )) 
                running_loss = 0.0
        # save weight
        #print((len(train_dataloader)/batch_size))
        if len(total_loss) != 0:
            total_avg_loss = sum(total_loss)/(len(total_loss)*batch_size)
        acc = 100*correct/total
        if total_avg_loss != 0:
            print("total average loss : %.3f" %(total_avg_loss))
        else :
            print("total loss :" ,total_loss)
        print("==epoch %d ==  train acc : %.4f" %(epoch,acc))
        TensorBoardWriter.add_scalar("Model1/ACC_Train",acc,epoch)
        TensorBoardWriter.add_scalar("Model1/LOSS_Train",total_avg_loss,epoch)
        acc = eval(target_model,test_dataloader,epoch,loss_fn,batch_size,
                   f4f,error_info,log_file,TensorBoardWriter)
        
        offset_info.append(target_model.get_layer(34))
        #torch.save(f4f.get_f4f_weight(), 
        #       retrain_model_path+"%s~%s_pkt_err_f4f_epoch_%s_acc_%.4f_loss_%.4f.pt"
        #       %(str(error_idx).zfill(3),str(error_idx+num_error).zfill(3),
        #        str(epoch+1).zfill(2),acc,total_avg_loss))    
    return first_feature,first_label,offset_info
                

In [12]:

#optimizer = torch.optim.SGD(param_list,lr=0.01,weight_decay=1e-4)
first_feature = []
first_label = []
offset_info = []
f = open(log_file,"w")
print("3 condconv")
f.close()
writer.flush()
#header.hook_register(vgg16_bn)
target_model = Target_model(vgg16_bn).to(device)

max_epoch = 90
tmp= training(f4f,target_model,original_model,
                  train_dataloader,test_dataloader,batch_size,
                  log_file,writer,retrain_model_path,
                  loss_fn,optimizer,
                  num_error,max_epoch,True)
        # tmp : first_feature,first_label,offset_info
writer.close()
first_feature.append(tmp[0])
first_label.append(tmp[1])
offset_info.append(tmp[2])

3 condconv
.

/media/2/hwbae0326/anaconda3/envs/local_pytorch/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


.........
[2,    99] loss: 0.030111
..........
[2,   199] loss: 0.030094
..........
[2,   299] loss: 0.030107
..........
[2,   399] loss: 0.030069
..........
[2,   499] loss: 0.030072
..........
[2,   599] loss: 0.030001
..........
[2,   699] loss: 0.030073
..........
[2,   799] loss: 0.030109
..........
[2,   899] loss: 0.030048
..........
[2,   999] loss: 0.030071
total average loss : 0.030
==epoch 1 ==  train acc : 0.1023
======eval start=======
step : 200 / 391 acc : 0.102

Eval acc of model on imagenet : 0.1000 %, Loss : 0.0301
======eval  end ======
..........
[3,    99] loss: 0.030119
..........
[3,   199] loss: 0.030088
..........
[3,   299] loss: 0.030095
..........
[3,   399] loss: 0.029995
..........
[3,   499] loss: 0.030130
..........
[3,   599] loss: 0.030021
..........
[3,   699] loss: 0.030089
..........
[3,   799] loss: 0.030012
..........
[3,   899] loss: 0.030052
..........
[3,   999] loss: 0.030160
total average loss : 0.030
==epoch 2 ==  train acc : 0.1094
======ev